In [52]:
from urllib.request import urlopen
from datetime import datetime, timedelta

def printToTelegram(textToTelegram) :
    print(textToTelegram)

lastfmUser='Anastasia20009'
daysDelta = 3
hoursDelta = 0
minListens = 3

artistList = dict()
timeDelay = timedelta(days=daysDelta,hours=hoursDelta)
today = datetime.utcnow()
overtime = False
noMoreTracks = False

def pageCacher(pageType,pageId,url) :
    htmlList = []
    if pageType == 'eventPage' :
        cacheFileName = pageId + '_lastfm.html'                   #'/cache/concerts/'
        try :         
            cacheHandle = open(cacheFileName, 'r', encoding="utf-8")
            htmlText = cacheHandle.read()
            htmlList = htmlText.splitlines()
            print( pageId + 'event page found in cache..',end='')
            cacheHandle.close()
        except FileNotFoundError:
            print('Downloading {} event page...'.format(pageId), end = '')            
    elif pageType == 'libraryPage' :
        cacheFileName = '{}_page_{}.html'.format(pageId, pageNum) #/cache/library/
        try :
            cacheHandle = open(cacheFileName, 'r', encoding="utf-8")
            htmlText = cacheHandle.read()  
            htmlList = htmlText.splitlines()
            print( 'Library page {} found in cache..'.format(pageNum),end='')  
            cacheHandle.close()
        except FileNotFoundError:
            print('Downloading {} library page...'.format(pageNum), end = '')
    if htmlList == [] :
        htmlByte = urlopen(url).read()
        htmlText = htmlByte.decode()   
        htmlList = htmlText.splitlines()
        cacheHandle = open(cacheFileName,'w', encoding="utf-8")
        cacheHandle.write(htmlText)
        cacheHandle.close()
        print('Done')
    return htmlList

def parserLibrary(htmlList) :
    htmlIterator = iter(htmlList)
    line = next(htmlIterator)
    try :
        for i in range(0,50) :
            while 'class="chartlist-artist' not in line : line = next(htmlIterator)
            while 'title=' not in line : line = next(htmlIterator)
            artist = line.split('"')[1]
            while 'chartlist-timestamp--lang-en' not in line : line = next(htmlIterator)
            while 'span title' not in line : line = next(htmlIterator)        
            timeStr = line.split('"')[1]
            timeStrFormatted = datetime.strptime(timeStr,"%A %d %b %Y, %I:%M%p")
            if (today - timeStrFormatted) > timeDelay : 
                overtime = True
                print('Parsed. timeDelay reached on page ',pageNum)
                return overtime
            else :
                artistList[artist] = artistList.get(artist,0) + 1 #сделать форматирование по регистру АБВ->Абв
    except StopIteration :
        noMoreTracks = True
        print('No more tracks')
        return noMoreTracks
    print('Parsed')

In [53]:
pageNum = 0
stopParsing = False

artistList = {}
while not stopParsing:
    if pageNum >= 100 : 
        print('Too many pages')
        break
    pageNum += 1
    lastfmUserUrl = 'https://www.last.fm/user/'+lastfmUser+'/library?page='+str(pageNum)   
    htmlList = pageCacher(pageType='libraryPage',pageId=lastfmUser,url=lastfmUserUrl)
    stopParsing = parserLibrary(htmlList)

provedArtistList = list()
for actArtist, listens in artistList.items() :
    if listens >= minListens :
        provedArtistList.append(actArtist)
print ('provedArtistList:', provedArtistList)
print ('Artist proved:',len(provedArtistList))

Library page 1 found in cache..Parsed
Library page 2 found in cache..Parsed
Library page 3 found in cache..Parsed. timeDelay reached on page  3
provedArtistList: ['Shortparis', 'Nautilus Pompilius', 'Morgenshtern', 'Corpse', 'Комиссар', 'INSTASAMKA', 'Halsey', 'Ляпис Трубецкой', 'Chills', 'Måneskin']
Artist proved: 10


In [55]:
import urllib.parse
artistEventBasket = dict()
eventBasket = dict()

def parserLastfmEvent(htmlList) :
    htmlIterator = iter(htmlList)
    line = next(htmlIterator)
    try :
        for i in range(0,100) :
            while 'class="events-list-item-date"' not in line : line = next(htmlIterator)
            eventTime = line.split('"')[5][:10]
            while 'class="events-list-item-venue--title"' not in line : line = next(htmlIterator)
            line = next(htmlIterator)
            eventVenue = line.strip()
            while 'class="events-list-item-venue--address"' not in line : line = next(htmlIterator)
            line = next(htmlIterator)
            eventAddress = line.strip()
            artistEventBasket[eventArtist+'-in-'+eventTime]={
                                         'eventartist':eventArtist,
                                         'eventtime':eventTime,
                                         'eventvenue':eventVenue,
                                         'eventcity':eventAddress.split(', ')[0],
                                         'eventcountry':eventAddress.split(', ')[1]}
    except StopIteration :
        if artistEventBasket == {} :
            print('No concerts from ',eventArtist)
            return
        else :
            print('Done with ',eventArtist)
            return
        
for eventArtist in provedArtistList :
    lastfmEventUrl = 'https://www.last.fm/music/' + urllib.parse.quote(eventArtist,safe='') + '/+events'
    htmlList = pageCacher(pageType='eventPage',pageId=eventArtist,url=lastfmEventUrl)
    parserLastfmEvent(htmlList)
    if artistEventBasket != {} :
        eventBasket[eventArtist] = artistEventBasket
        artistEventBasket = {}
    
def prettyThree(threeIndentDict, indent=0):
    prettyList = list()
    prettyString = str()
    for key1, value1 in threeIndentDict.items():
        prettyList.append(str(key1)+'\n')
        for key2, value2 in value1.items():
            prettyList.append(' ' * indent + str(key2) + '\n')
            for k, v in value2.items() :
                prettyList.append(' ' * 2 * indent + k + ' : ' + v + '\n')
    prettyString = prettyString.join(prettyList)
    printToTelegram(prettyString)
                
prettyThree(eventBasket,4)
print('Event basket was sent to Telegram')

Shortparisevent page found in cache..Done with  Shortparis
Nautilus Pompiliusevent page found in cache..No concerts from  Nautilus Pompilius
Morgenshternevent page found in cache..Done with  Morgenshtern
Corpseevent page found in cache..No concerts from  Corpse
Комиссарevent page found in cache..No concerts from  Комиссар
INSTASAMKAevent page found in cache..No concerts from  INSTASAMKA
Halseyevent page found in cache..No concerts from  Halsey
Ляпис Трубецкойevent page found in cache..No concerts from  Ляпис Трубецкой
Chillsevent page found in cache..No concerts from  Chills
Måneskinevent page found in cache..Done with  Måneskin
Shortparis
    Shortparis-in-2021-12-04
        eventartist : Shortparis
        eventtime : 2021-12-04
        eventvenue : Main Stage
        eventcity : Moscow
        eventcountry : Russian Federation
    Shortparis-in-2021-11-05
        eventartist : Shortparis
        eventtime : 2021-11-05
        eventvenue : BACKSTAGE
        eventcity : Тула
        e

In [135]:
def prettyThree(threeIndentDict, indent=0):
    prettyList = list()
    prettyString = str()
    for key1, value1 in threeIndentDict.items():
        prettyList.append(str(key1)+'\n')
        for key2, value2 in value1.items():
            prettyList.append(' ' * indent + str(key2) + '\n')
            for k, v in value2.items() :
                prettyList.append(' ' * 2 * indent + k + ' : ' + v + '\n')
    prettyString = prettyString.join(prettyList)
    printToTelegram(prettyString)
                
prettyThree(eventBasket, 4)

Операция Пластилин
    Операция Пластилин-in-2021-09-25
        eventartist : Операция Пластилин
        eventtime : 2021-09-25
        eventvenue : Махно-Паб
        eventcity : Днепр
        eventcountry : Ukraine
    Операция Пластилин-in-2022-07-01
        eventartist : Операция Пластилин
        eventtime : 2022-07-01
        eventvenue : Аэродром «Левцово»
        eventcity : Ярославская область
        eventcountry : Russian Federation
    Операция Пластилин-in-2022-07-14
        eventartist : Операция Пластилин
        eventtime : 2022-07-14
        eventvenue : Улетай поле
        eventcity : село Лагуново
        eventcountry : Russian Federation



In [20]:
htmlList

'\n\n\n\n\n\n<!doctype html>\n<html\n    lang="en"\n    class="\n        no-js\n        playbar-masthead-release-shim\n        youtube-provider-not-ready\n    "\n>\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge"><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UwYPV15QGwYFXFlXDgU=",licenseKey:"0ed0ce50b0",applicationID:"5588594"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{s.console&&console.log(t)}catch(e){}}var i,o=t("ee"),a=t(25),s={};try{i=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,i.indexOf("dev")!==-1&&(s.dev=!0),i.indexOf("nr_dev")!==-1&&(s.nrD